# Conjunto de dados

aXXX

# Processamento dos dados

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.4 MB/s eta 0:00:00


In [ ]:
pip install dalex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dalex: filename=dalex-1.7.2-py3-none-any.whl size=1042898 sha256=fe4fb58fa9d47093d85d944fa9937b19e7b5497a43545d9899ba4739de13fce6
  Stored in directory: /root/.cache/pip/wheels/cd/5e/ec/848120d5b913158656c0f8ef316233dd3356effa64aef19fba
Successfully built dalex


In [ ]:
!pip install aif360 numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px



from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score
from sklearn import metrics

from io import StringIO
import sys

import re
import sys
from io import StringIO

import dalex as dx

In [ ]:
law = pd.read_csv("law_school_clean.csv")

In [ ]:
law.columns

Index(['decile1b', 'decile3', 'lsat', 'ugpa', 'zfygpa', 'zgpa', 'fulltime',
       'fam_inc', 'male', 'tier', 'race', 'pass_bar'],
      dtype='object')

In [ ]:
law.shape

(20798, 12)

In [ ]:
law.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race,pass_bar
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,White,1.0
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,White,1.0
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,White,1.0
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,White,1.0
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,White,1.0


In [ ]:
law["male"].value_counts()

,count
male,
1.0,11675
0.0,9123


In [ ]:
law["pass_bar"].value_counts()

,count
pass_bar,
1.0,18505
0.0,2293


In [ ]:
law=law.drop(["race"], axis=1)

In [ ]:
law.rename(columns={"male": "sex"},  inplace=True)

In [ ]:
law["sex"].value_counts()

,count
sex,
1.0,11675
0.0,9123


In [ ]:
Categorical=law.select_dtypes(include='object').columns.tolist()
Categorical

[]

In [ ]:
Numerical=law.select_dtypes(exclude='object').columns.tolist()
Numerical

['decile1b',
 'decile3',
 'lsat',
 'ugpa',
 'zfygpa',
 'zgpa',
 'fulltime',
 'fam_inc',
 'sex',
 'tier',
 'pass_bar']

# Primeira etapa do experimento

* dataset do compas
* vários algoritmos
* várias métricas de **fairness**

Fairness by unawareness ou justiça através do desconhecimento é um conceito ingênuo de  justiça em que um algoritmo justo é definido quando não usa nenhum atributo protegido no processo de treinamento. Então não existe essa ideia de treinar sem o atributo protegido.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis =1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
categorical_features = []
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_features = ['decile1b',
 'decile3',
 'lsat',
 'ugpa',
 'zfygpa',
 'zgpa',
 'fulltime',
 'fam_inc',
 'sex',
 'tier'


]
numerical_transformer = Pipeline(steps=[
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
])

### Treinando para vários algoritmos

In [ ]:
  clf_forest = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=123))
  ])

  clf_logreg = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression())
  ])

  clf_gb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GradientBoostingClassifier(random_state=123))
  ])

  clf_xgb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', XGBClassifier(random_state=123))
  ])

  clf_lgbm = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LGBMClassifier(random_state=123))
  ])

  clf_catboost = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', CatBoostClassifier(random_state=123, verbose=0))
  ])

  clf_ridge = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RidgeClassifier())
  ])

  clf_sgd = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SGDClassifier(random_state=123))
  ])

  clf_svc = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SVC(random_state=123))
  ])

  clf_knn = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', KNeighborsClassifier())
  ])

  clf_nb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GaussianNB())
  ])

  clf_mlp = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', MLPClassifier(random_state=123))
  ])

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    'SVC': clf_svc,
    'KNeighbors': clf_knn,
    'GaussianNB': clf_nb,
    'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Avaliando para várias métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)
    relatorio = {'accuracy': accuracy_score(y_test, y_pred), 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy                model
0   0.899519        Random Forest
1   0.899519  Logistic Regression
2   0.898878    Gradient Boosting
3   0.892788              XGBoost
4   0.895353             CatBoost
5   0.892788     Ridge Classifier
6   0.895192       SGD Classifier
7   0.897756                  SVC
8   0.885417           KNeighbors
9   0.811218           GaussianNB
10  0.896955        MLPClassifier


In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy    recall  precision       auc        f1                model
0   0.899519  0.982005   0.911932  0.605204  0.945672        Random Forest
1   0.899519  0.978586   0.914564  0.617404  0.945492  Logistic Regression
2   0.898878  0.977506   0.914786  0.618328  0.945107    Gradient Boosting
3   0.892788  0.966889   0.917207  0.628393  0.941393              XGBoost
4   0.895353  0.972107   0.915593  0.621486  0.943004             CatBoost
5   0.892788  0.999460   0.892926  0.512175  0.943194     Ridge Classifier
6   0.895192  0.995321   0.898035  0.537924  0.944179       SGD Classifier
7   0.897756  0.987583   0.906059  0.577247  0.945066                  SVC
8   0.885417  0.963290   0.912858  0.607560  0.937396           KNeighbors
9   0.811218  0.824366   0.957767  0.764306  0.886074           GaussianNB
10  0.896955  0.975886   0.914194  0.615322  0.944033        MLPClassifier


In [ ]:
#df_resultados.to_excel(r"Performance padrão dos algoritmos sem balanceamento.xlsx", index=False)

### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers = {}

for model_name, model in trained_models.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.153, mean = 0.889, max = 0.996
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.996, mean = 0.00203, max = 0.847
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

Para automatizar o processo de concatenar os resultados de desempenho de todos os modelos explicados com DALEX, irei iterar sobre os explicadores armazenados em um dicionário e concatenar seus resultados.

In [ ]:
performance_results = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name
    performance_results.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results = pd.concat(performance_results, ignore_index=True)

o método model_perfomance realiza o cálculo de medidas de desempenho do modelo


In [ ]:
protected = X_test.sex

In [ ]:
model_fairness_results = {}

for model_name, explainer in explainers.items():
    fairness_result = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results[model_name] = fairness_result

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
fairness_checks = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result in model_fairness_results.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks = pd.DataFrame(fairness_checks)

# Exibir o DataFrame com os resultados
print(all_fairness_checks)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC      PPV       FPR       STP
0.0  0.994919  0.979075  0.98259  0.921642  0.981386
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.994919, FPR=0.979075, STP=0.98259
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.991853  0.979075  0.984799  0.894402  0.976116
Métricas de viés detectadas para 

Este código captura e processa a saída do método fairness_check() para cada modelo, extrai as métricas relevantes e armazena essas informações em um DataFrame para análise. Isso facilita a visualização e comparação dos resultados de diferentes modelos em termos de métricas de viés detectadas.

In [ ]:
all_fairness_checks.head()

,Modelo,Num Biases Detected,TPR,ACC,PPV,FPR,STP
0,Random Forest,None,1.25,0.0,0.994919,0.979075,0.982590
1,Logistic Regression,None,1.25,0.0,0.991853,0.979075,0.984799
2,Gradient Boosting,None,1.25,0.0,0.989817,0.974697,0.981562
3,XGBoost,None,1.25,0.0,0.992784,0.982222,0.983766
4,CatBoost,None,1.25,0.0,0.995893,0.983370,0.984816


# Segunda etapa do experimento - Balancear os dados para mitigar o viés

* dataset do compas
* balanceamento dos dados no treinamento
* várias técnicas de balanceamento dos dados
* vários algoritmos
* várias métricas de **fairness**

### Abordagem 1 - Embaralhar a coluna de sexo desbalanceada

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis =1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
X_train_abordagemUm = X_train.copy().reset_index(drop=True)

X_train_abordagemUm["sex"] = X_train["sex"].sample(frac=1).reset_index(drop=True)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_abordagemUm, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemUm = pd.DataFrame(resultados)
print(df_resultados_abordagemUm)


    accuracy    recall  precision       auc        f1                model
0   0.898718  0.982185   0.911033  0.600902  0.945272        Random Forest
1   0.897756  0.978046   0.913292  0.611278  0.944560  Logistic Regression
2   0.898718  0.977506   0.914632  0.617596  0.945024    Gradient Boosting
3   0.890705  0.966349   0.915601  0.620802  0.940291              XGBoost
4   0.895353  0.974087   0.914049  0.614423  0.943114             CatBoost
5   0.892468  0.999640   0.892513  0.510069  0.943044     Ridge Classifier
6   0.896474  0.984344   0.907281  0.582948  0.944243       SGD Classifier
7   0.897115  0.987763   0.905327  0.573677  0.944750                  SVC
8   0.887340  0.964909   0.913458  0.610566  0.938479           KNeighbors
9   0.810417  0.823286   0.957915  0.764498  0.885512           GaussianNB
10  0.899038  0.980385   0.912716  0.608787  0.945341        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.154, mean = 0.889, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.996, mean = 0.00185, max = 0.845
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV      FPR       STP
0.0  0.997965  0.976872  0.978261  0.96478  0.988601
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.997965, FPR=0.976872, STP=0.978261
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
     TPR      ACC       PPV       FPR       STP
0.0  1.0  0.98232  0.979393  0.953488  0.989562
Métricas de viés detectadas para Logistic Re

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [ ]:
fairness_result_balanceamento.plot()

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 2 - Duplicar os dados de treinamento com a classe oposta

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis =1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
df2 = X_train.copy()

df2["sex"] = df2["sex"].replace({1:0, 0: 1})
X_train2 = pd.concat([df2, X_train])
print(X_train2.shape, X_train.shape)
X_train2['sex'].value_counts()

(29116, 10) (14558, 10)


,count
sex,
1.0,14558
0.0,14558


In [ ]:
y_train2 = np.concatenate([y_train, y_train])

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados duplicados no treinamento com o sexo (classe) oposto
trained_models_duplicados = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train2, y_train2)
    trained_models_duplicados[model_name] = model
    print(f"Treinando {model_name} com dados de treinamento duplicados")

Treinando Random Forest com dados de treinamento duplicados
Treinando Logistic Regression com dados de treinamento duplicados
Treinando Gradient Boosting com dados de treinamento duplicados
Treinando XGBoost com dados de treinamento duplicados
Treinando CatBoost com dados de treinamento duplicados
Treinando Ridge Classifier com dados de treinamento duplicados
Treinando SGD Classifier com dados de treinamento duplicados
Treinando SVC com dados de treinamento duplicados
Treinando KNeighbors com dados de treinamento duplicados
Treinando GaussianNB com dados de treinamento duplicados
Treinando MLPClassifier com dados de treinamento duplicados


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_duplicados.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemDois = pd.DataFrame(resultados)
print(df_resultados_abordagemDois)


    accuracy    recall  precision       auc        f1                model
0   0.899519  0.981645   0.912207  0.606489  0.945653        Random Forest
1   0.898397  0.978046   0.913906  0.614206  0.944889  Logistic Regression
2   0.899359  0.977866   0.914969  0.619240  0.945372    Gradient Boosting
3   0.889423  0.963829   0.916339  0.623935  0.939484              XGBoost
4   0.892949  0.969948   0.914955  0.618210  0.941649             CatBoost
5   0.892468  0.999640   0.892513  0.510069  0.943044     Ridge Classifier
6   0.891667  0.979665   0.906276  0.577680  0.941543       SGD Classifier
7   0.898237  0.987583   0.906508  0.579443  0.945310                  SVC
8   0.884936  0.962030   0.913378  0.609858  0.937073           KNeighbors
9   0.810737  0.823466   0.958124  0.765320  0.885706           GaussianNB
10  0.896955  0.973187   0.916300  0.624954  0.943887        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_dois = {}

for model_name, model in trained_models_duplicados.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_dois[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.148, mean = 0.889, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.997, mean = 0.00185, max = 0.844
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento_dois = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_dois.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_dois.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_dois = pd.concat(performance_results_balanceamento_dois, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_dois = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_dois.items():
    fairness_result_balanceamento_dois = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento_dois[model_name] = fairness_result_balanceamento_dois

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_dois = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_dois is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_dois.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_dois.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_dois = pd.DataFrame(fairness_checks_balanceamento_dois)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_dois)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.994919  0.979075  0.980435  0.935849  0.982402
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.994919, FPR=0.979075, STP=0.980435
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.997957  0.983425  0.982628  0.923077  0.985401
Métricas de viés detectadas pa

In [ ]:
# Gráfico de Barras: Comparação do Número de Métricas de Viés Detectadas por Modelo
fig = px.bar(
    all_fairness_checks_balanceamento_dois,
    x='Modelo',
    y='Num Biases Detected',
    title="Número de Métricas de Viés Detectadas por Modelo",
    color='Num Biases Detected',  # Colore as barras com base no valor
    color_continuous_scale='viridis',
    labels={"Num Biases Detected": "Número de Métricas de Viés", "Modelo": "Modelo"}
)

# Ajuste de layout para legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    title_x=0.5,  # Centraliza o título
    yaxis_title="Número de Métricas de Viés",
    xaxis_title="Modelo"
)

# Exibindo o gráfico
fig.show()

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_dois.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_dois.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier

Found NaN's or 0's for models: {'MLPClassifier'}
It is advisable to check 'metric_ratios'


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 3 - Subamostragem da Classe Majoritária

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis=1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

# Aplicar subamostragem da classe majoritária
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_subamostragem = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_res, y_train_res)
    trained_models_subamostragem[model_name] = model
    print(f"Treinado {model_name} com subamostragem")

Treinado Random Forest com subamostragem
Treinado Logistic Regression com subamostragem
Treinado Gradient Boosting com subamostragem
Treinado XGBoost com subamostragem
Treinado CatBoost com subamostragem
Treinado Ridge Classifier com subamostragem
Treinado SGD Classifier com subamostragem
Treinado SVC com subamostragem
Treinado KNeighbors com subamostragem
Treinado GaussianNB com subamostragem
Treinado MLPClassifier com subamostragem


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_subamostragem.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemTres = pd.DataFrame(resultados)
print(df_resultados_abordagemTres)


    accuracy    recall  precision       auc        f1                model
0   0.783494  0.779594   0.972222  0.797672  0.865318        Random Forest
1   0.786538  0.784085   0.971079  0.795460  0.867621  Logistic Regression
2   0.787340  0.785522   0.970484  0.793950  0.868262    Gradient Boosting
3   0.763942  0.764505   0.963332  0.761896  0.852479              XGBoost
4   0.784295  0.783187   0.969104  0.788324  0.866283             CatBoost
5   0.769231  0.761990   0.973606  0.795557  0.854897     Ridge Classifier
6   0.779808  0.781390   0.965165  0.774053  0.863609       SGD Classifier
7   0.772917  0.767200   0.972450  0.793704  0.857717                  SVC
8   0.761538  0.760014   0.965321  0.767080  0.850452           KNeighbors
9   0.729167  0.715466   0.974077  0.778981  0.824979           GaussianNB
10  0.783814  0.782468   0.969292  0.788708  0.865918        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_tres = {}

for model_name, model in trained_models_subamostragem.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_tres[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0331, mean = 0.667, max = 0.991
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.987, mean = 0.226, max = 0.963
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento_tres = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_tres.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_tres.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_tres = pd.concat(performance_results_balanceamento_tres, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_tres = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_tres.items():
    fairness_result_balanceamento_tres = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento_tres[model_name] = fairness_result_balanceamento_tres

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_tres = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_tres is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_tres.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_tres.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_tres = pd.DataFrame(fairness_checks_balanceamento_tres)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_tres)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV      FPR       STP
0.0  0.985969  0.992366  0.996917  0.86802  0.962861
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.985969, FPR=0.992366, STP=0.996917
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.947631  0.957553  0.994861  0.873786  0.927419
Métricas de viés detectadas para

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_tres.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_tres.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 4 - Utilização da biblioteca SMOTE para criar exemplos sintéticos da classe minoritária - KMeans-SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# Supomos que 'compas' seja seu DataFrame original
categorical_columns = []
non_categorical_columns = law.drop(columns=categorical_columns)  # Selecionar as colunas restantes

# Inicializar o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Aplicar o encoder nas colunas categóricas
encoded_data = encoder.fit_transform(law[categorical_columns]).toarray()  # Convertendo para array denso

# Criar um DataFrame com as colunas codificadas
encoded_df = pd.DataFrame(encoded_data,
                          columns=encoder.get_feature_names_out(categorical_columns),
                          index=law.index)  # Preserva os índices originais

# Concatenar o DataFrame codificado com as colunas não categóricas
df_encoded = pd.concat([non_categorical_columns, encoded_df], axis=1)

# Exibir o DataFrame final
df_encoded.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,sex,tier,pass_bar
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,1.0
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,1.0
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,1.0
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,1.0
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,1.0


In [ ]:
law_encoded = df_encoded

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law_encoded.drop(["pass_bar"], axis=1),
                                                    law_encoded.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_group = smote.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_smote['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)


# Verifica resultado
print(y_income.value_counts())
print(X_train_smote['sex'].value_counts())


1
1    14622
0    14622
Name: count, dtype: int64
sex
0    14622
1    14622
Name: count, dtype: int64


O SMOTE cria exemplos sintéticos da classe minoritária, aumentando o conjunto de dados de forma mais natural.

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_smote = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_smote, y_income)
    trained_models_smote[model_name] = model
    print(f"Treinando {model_name} com smote")

Treinando Random Forest com smote
Treinando Logistic Regression com smote
Treinando Gradient Boosting com smote
Treinando XGBoost com smote
Treinando CatBoost com smote
Treinando Ridge Classifier com smote
Treinando SGD Classifier com smote
Treinando SVC com smote
Treinando KNeighbors com smote
Treinando GaussianNB com smote
Treinando MLPClassifier com smote


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_smote.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemQuatro = pd.DataFrame(resultados)
print(df_resultados_abordagemQuatro)


    accuracy    recall  precision       auc        f1                model
0   0.826122  0.839950   0.960164  0.775844  0.896043        Random Forest
1   0.782692  0.780672   0.969873  0.790039  0.865048  Logistic Regression
2   0.861218  0.892042   0.949340  0.749142  0.919800    Gradient Boosting
3   0.890064  0.954374   0.924804  0.656236  0.939356              XGBoost
4   0.898237  0.965870   0.923566  0.652326  0.944244             CatBoost
5   0.762179  0.753727   0.973776  0.792911  0.849737     Ridge Classifier
6   0.771795  0.767379   0.970688  0.787850  0.857143       SGD Classifier
7   0.787981  0.789653   0.966579  0.781899  0.869204                  SVC
8   0.800962  0.821089   0.948931  0.727781  0.880393           KNeighbors
9   0.729968  0.716544   0.973877  0.778777  0.825624           GaussianNB
10  0.816987  0.839231   0.949787  0.736109  0.891093        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_quatro = {}

for model_name, model in trained_models_smote.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_quatro[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0631, mean = 0.723, max = 0.994
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.993, mean = 0.169, max = 0.926
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento_quatro = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_quatro.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_quatro.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_quatro = pd.concat(performance_results_balanceamento_quatro, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_quatro = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_quatro.items():
    fairness_result_balanceamento_quatro = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento_quatro[model_name] = fairness_result_balanceamento_quatro

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_quatro = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_quatro is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_quatro.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])

        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_quatro.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_quatro = pd.DataFrame(fairness_checks_balanceamento_quatro)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_quatro)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR  ACC       PPV      FPR       STP
0.0  1.004773  1.0  0.988601  1.03169  0.989796
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=1.004773, FPR=1.0, STP=0.995881
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC      PPV       FPR       STP
0.0  0.984733  0.987294  0.99076  1.035533  0.968407
Métricas de viés detectadas para Logistic Regress

 graficos das metricas de fairness do dalex e avaliação para algunos modelos

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_quatro.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_quatro.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 5 - FairSmote (Implementação de um algoritmo que remove rótulos tendenciosos e rebalanceia as distribuições internas de forma que, com base no atributo sensível, os exemplos sejam iguais tanto nas classes positivas quanto negativas.)

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
from sklearn.neighbors import NearestNeighbors as NN

class smote(object):
  def __init__(self, pd_data, neighbor=5,r=2 ,up_to_num=[],auto=True):
    """
    :param pd_data: panda.DataFrame, the last column must be class label
    :param neighbor: num of nearst neighbors to select
    :param up_to_num: size of minorities to over-sampling
    :param up_to_max: if up_to_num is not supplied, all minority classes will
                      be over-sampled as much as majority class
    :return panda.DataFrame smoted data
    """
    self.set_data(pd_data)
    self.auto = auto
    self.neighbor = neighbor
    self.up_to_max = False
    self.up_to_num = up_to_num
    self.r = r
    self.label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #if up_to_num:
    #  label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #  if label_num - 1 != len(up_to_num):
    #    raise ValueError(
    #      "should set smoted size for " + str(label_num - 1) + " minorities")
    #  self.up_to_num = up_to_num
    #else:
    #  self.up_to_max = True

  def set_data(self, pd_data):
    if not pd_data.empty:# and isinstance(
        #pd_data.ix[:, pd_data.columns[-1]].values[0], str):
      self.data = pd_data
    else:
      raise ValueError(
        "The last column of pd_data should be string as class label")

  def get_majority_num(self):
    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    return majority_num

  def run(self):
    """
    run smote
    """

    def get_ngbr(data_no_label, knn):
      rand_sample_idx = random.randint(0, len(data_no_label) - 1)
      rand_sample = data_no_label[rand_sample_idx]
      distance, ngbr = knn.kneighbors(rand_sample.reshape(1, -1))
      # while True:
      rand_ngbr_idx = random.randint(0, len(ngbr))
      #   if distance[rand_ngbr_idx] == 0:
      #     continue  # avoid the sample itself, where distance ==0
      #   else:
      return data_no_label[rand_ngbr_idx], rand_sample

    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    for label, num in labelCont.items():
      if num < majority_num:
        to_add = majority_num - num
        last_column = self.data[self.data.columns[-1]]
        data_w_label = self.data.loc[last_column == label]
        data_no_label = data_w_label[self.data.columns[:-1]].values
        if len(data_no_label) < self.neighbor:
          num_neigh = len(data_no_label) # void # of neighbors >= sample size
        else:
          num_neigh = self.neighbor
        knn = NN(n_neighbors=num_neigh,p=self.r,algorithm='ball_tree').fit(data_no_label)
        if self.auto:
          to_add = to_add
        else:
          to_add = self.up_to_num
        for _ in range(to_add):
          rand_ngbr, sample = get_ngbr(data_no_label, knn)
          new_row = []
          for i, one in enumerate(rand_ngbr):
            gap = random.random()
            new_row.append(max(0, sample[i] + (
            sample[i] - one) * gap))  # here, feature vlaue should >=0
          new_row.append(label)
          total_data.append(new_row)
    return pd.DataFrame(total_data)

In [ ]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = smote(df)
    df = smt.run()
    df.columns = cols
    return df

In [ ]:
dataset_orig_train, dataset_orig_test = train_test_split(law_encoded, test_size=0.2, shuffle = True)

In [ ]:
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'pass_bar'], dataset_orig_train['pass_bar']
X_test, y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'pass_bar'], dataset_orig_test['pass_bar']

In [ ]:
train_df = X_train
train_df['pass_bar'] = y_train

In [ ]:
train_df = apply_smote(train_df)

In [ ]:
y_train = train_df.pass_bar
X_train = train_df.drop('pass_bar', axis = 1)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemCinco = pd.DataFrame(resultados)
print(df_resultados_abordagemCinco)


    accuracy    recall  precision       auc        f1                model
0   0.899038  0.985147   0.909046  0.593230  0.945568        Random Forest
1   0.849279  0.922495   0.909478  0.589256  0.915940  Logistic Regression
2   0.899519  0.978126   0.914877  0.620354  0.945445    Gradient Boosting
3   0.889183  0.966244   0.914154  0.615507  0.939477              XGBoost
4   0.891106  0.969484   0.913486  0.612751  0.940652             CatBoost
5   0.884615  0.981366   0.898393  0.541011  0.938049     Ridge Classifier
6   0.880529  0.981096   0.894828  0.523371  0.935978       SGD Classifier
7   0.901202  0.985687   0.910679  0.601159  0.946700                  SVC
8   0.874519  0.949500   0.913010  0.608229  0.930898           KNeighbors
9   0.799519  0.846611   0.921788  0.632277  0.882601           GaussianNB
10  0.895192  0.978126   0.910737  0.600660  0.943229        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 4160 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 4160 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.223, mean = 0.843, max = 0.926
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.926, mean = 0.0467, max = 0.777
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.995947  0.980154  0.982533  0.927624  0.983539
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.995947, FPR=0.980154, STP=0.982533
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR      ACC       PPV       FPR       STP
0.0  1.006522  0.97669  0.967497  1.107649  1.010011
Métricas de viés detectadas para

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 6 - ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis=1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

# Aplicar ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_group = adasyn.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_adasyn['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_adasyn['sex'].value_counts())


1
1    14727
0    14534
Name: count, dtype: int64
sex
0    14690
1    14571
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_adasyn, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.817147  0.827555   0.962197  0.779305  0.889812        Random Forest
1   0.773077  0.768637   0.970955  0.789222  0.858031  Logistic Regression
2   0.864103  0.896713   0.948148  0.745533  0.921713    Gradient Boosting
3   0.891987  0.959224   0.922758  0.647517  0.940638              XGBoost
4   0.897917  0.963535   0.925147  0.659331  0.943951             CatBoost
5   0.757853  0.747979   0.974719  0.793752  0.846427     Ridge Classifier
6   0.751603  0.741153   0.974262  0.789596  0.841869       SGD Classifier
7   0.770673  0.765942   0.970856  0.787874  0.856311                  SVC
8   0.794551  0.816059   0.946261  0.716350  0.876350           KNeighbors
9   0.717949  0.701635   0.975281  0.777266  0.816130           GaussianNB
10  0.791506  0.804742   0.954410  0.743382  0.873209        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0905, mean = 0.712, max = 0.996
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.991, mean = 0.18, max = 0.896
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data       

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  1.001209  0.997555  0.988625  1.037879  0.985751
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=1.001209, FPR=0.997555, STP=0.988625
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV      FPR       STP
0.0  1.017038  1.009091  0.987705  1.22807  1.002837
Métricas de viés detectadas para

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 7 - Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis=1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

# Agora balanceia os grupos combinados (ex: mulher_<=50K, homem_>50K, etc.)
ros = RandomOverSampler(random_state=42)
X_train_ros, y_group = ros.fit_resample(X_train, group)

# Separar novamente income e sex
#X_train_ros['sex'] = X_train_ros['sex']  # já está incluso
y_split = y_group.str.split('_', expand=True)
X_train_ros['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)            # target original: income

# Verifica resultado
print(y_income.value_counts())
print(X_train_ros['sex'].value_counts())


1
1    14622
0    14622
Name: count, dtype: int64
sex
0    14622
1    14622
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_ros, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.789103  0.788396   0.969516  0.791672  0.869626        Random Forest
1   0.778365  0.775103   0.970535  0.790226  0.861880  Logistic Regression
2   0.781090  0.779055   0.969595  0.788487  0.863944    Gradient Boosting
3   0.832372  0.860607   0.946651  0.729709  0.901581              XGBoost
4   0.826603  0.846057   0.954407  0.755867  0.896972             CatBoost
5   0.762981  0.754266   0.974246  0.794667  0.850258     Ridge Classifier
6   0.754968  0.746003   0.973055  0.787563  0.844535       SGD Classifier
7   0.770032  0.764325   0.971905  0.790781  0.855706                  SVC
8   0.805769  0.831507   0.944116  0.712187  0.884241           KNeighbors
9   0.727724  0.713670   0.974252  0.778826  0.823847           GaussianNB
10  0.792949  0.801509   0.959776  0.761824  0.873532        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0519, mean = 0.681, max = 0.989
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.984, mean = 0.211, max = 0.948
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  1.003812  1.003807  0.993827  0.990291  0.984932
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=1.003812, FPR=1.003807, STP=0.993827
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR      ACC       PPV       FPR       STP
0.0  0.988447  0.98977  0.991786  1.005155  0.969529
Métricas de viés detectadas para

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 8 -  NearMiss (remove exemplos da classe majoritária de forma informada, ou seja, não aleatoriamente, mas baseada na distância entre os exemplos da classe majoritária e os da classe minoritária.)

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis=1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

near = NearMiss()
X_train_near, y_group = near.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_near['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_near['sex'].value_counts())


1
0    1572
1    1572
Name: count, dtype: int64
sex
0    1572
1    1572
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_near, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.413462  0.382432   0.905572  0.526283  0.537762        Random Forest
1   0.739744  0.760553   0.935691  0.664080  0.839080  Logistic Regression
2   0.328365  0.286869   0.878438  0.479244  0.432498    Gradient Boosting
3   0.276442  0.224897   0.862259  0.463860  0.356746              XGBoost
4   0.288942  0.237471   0.873184  0.476091  0.373394             CatBoost
5   0.745513  0.767738   0.935434  0.664701  0.843331     Ridge Classifier
6   0.840064  0.889348   0.928371  0.660870  0.908440       SGD Classifier
7   0.323878  0.279684   0.881653  0.484567  0.424656                  SVC
8   0.449199  0.433447   0.895030  0.506471  0.584049           KNeighbors
9   0.452724  0.447099   0.880750  0.473178  0.593113           GaussianNB
10  0.302244  0.253099   0.877882  0.480933  0.392917        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.417, max = 0.727
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.727, mean = 0.475, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data           

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  1.195467  1.163212  0.979235  1.174342  1.189655
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=1.195467, FPR=1.163212, STP=0.979235
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  1.013228  1.009498  0.989362  0.950339  0.997245
Métricas de viés detectadas pa

In [ ]:
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 9 - BorderlineSMOTE (Para superar a deficiência do SMOTE, ele identifica dois conjuntos de pontos — Ruído e Fronteira)

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis=1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

border = BorderlineSMOTE(random_state = 42, kind = 'borderline-2')
X_train_border, y_group = border.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_border['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_border['sex'].value_counts())


1
1    14622
0    14622
Name: count, dtype: int64
sex
0    14622
1    14622
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_border, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)

    accuracy    recall  precision       auc        f1                model
0   0.810256  0.817137   0.964793  0.785240  0.884847        Random Forest
1   0.805288  0.811209   0.964957  0.783762  0.881429  Logistic Regression
2   0.837660  0.855218   0.958333  0.773820  0.903844    Gradient Boosting
3   0.885897  0.941261   0.931556  0.684598  0.936383              XGBoost
4   0.890865  0.950602   0.928747  0.673666  0.939547             CatBoost
5   0.780449  0.777618   0.970410  0.790741  0.863383     Ridge Classifier
6   0.779006  0.776001   0.970350  0.789932  0.862362       SGD Classifier
7   0.791186  0.791629   0.968571  0.789574  0.871207                  SVC
8   0.832372  0.865278   0.942108  0.712728  0.902060           KNeighbors
9   0.752404  0.743848   0.972066  0.783514  0.842780           GaussianNB
10  0.814423  0.828094   0.958221  0.764716  0.888418        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0672, mean = 0.736, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.996, mean = 0.156, max = 0.933
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR      ACC       PPV       FPR       STP
0.0  0.989038  0.99139  0.994829  0.888889  0.966102
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.989038, FPR=0.99139, STP=0.994829
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  1.027431  1.017522  0.986612  1.188341  1.014765
Métricas de viés detectadas para 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 10 - Reweighting ( Reponderação sensível de classe) - Sample Weight


In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(law.drop(["pass_bar"], axis=1),
                                                    law.pass_bar,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
sex_train = X_train['sex']  # ou como for sua variável

# Rebalancear o sexo no treino usando sample_weight
sex_counts = sex_train.value_counts(normalize=True)
sex_weights = {sex: 1.0 / freq for sex, freq in sex_counts.items()}

# Cria o vetor de pesos
sample_weight = sex_train.map(sex_weights).values

#### Treinando para vários algoritmos

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    #'SVC': clf_svc,
    #'KNeighbors': clf_knn,
    #'GaussianNB': clf_nb,
    #'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train, classifier__sample_weight=sample_weight)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


   accuracy    recall  precision       auc        f1                model
0  0.906090  0.988504   0.913361  0.606436  0.949448        Random Forest
1  0.905128  0.983833   0.916039  0.618960  0.948727  Logistic Regression
2  0.902724  0.981498   0.915550  0.616306  0.947378    Gradient Boosting
3  0.896154  0.972157   0.916511  0.619808  0.943515              XGBoost
4  0.899359  0.977546   0.915391  0.615073  0.945448             CatBoost
5  0.893269  0.998743   0.894034  0.509772  0.943492     Ridge Classifier
6  0.892628  0.998922   0.893333  0.506148  0.943182       SGD Classifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6240 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6240 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7d9d543cbf60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.142, mean = 0.892, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.997, mean = 0.000363, max = 0.801
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data    

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1.0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.994955  0.985746  0.988017  0.895961  0.980493
Métricas de viés detectadas para Random Forest: TPR=1.25, ACC=0.0, PPV=0.994955, FPR=0.985746, STP=0.988017
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1.0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
          TPR       ACC       PPV       FPR       STP
0.0  0.987867  0.982456  0.990217  0.868256  0.971134
Métricas de viés detectadas pa

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



# Resultados Finais

In [ ]:
all_performance_results.to_excel(r"Performance dos algoritmos sem balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento.to_excel(r"Performance dos algoritmos com 1º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_dois.to_excel(r"Performance dos algoritmos com 2º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_tres.to_excel(r"Performance dos algoritmos com 3º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_quatro.to_excel(r"Performance dos algoritmos com 4º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_fairness_checks.to_excel(r"Resultado da avaliação se os modelos são justos - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento.to_excel(r"Resultado da avaliação se os modelos são justos com 1º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_dois.to_excel(r"Resultado da avaliação se os modelos são justos com 2º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_tres.to_excel(r"Resultado da avaliação se os modelos são justos com 3º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_quatro.to_excel(r"Resultado da avaliação se os modelos são justos com 4º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)